# Variant regulatory consequence analysis

This notebook introduces new dataset `Epiraction`, which contains the regulatory consequences for variants.

## Epiraction dataset ingesion

In [ ]:
from __future__ import annotations

from gentropy.common.session import Session
from pyspark.sql import functions as f

from manuscript_methods.datasets import LeadVariantEffect


In [5]:
session = Session(extended_spark_conf={"spark.driver.memory": "120G"})
dataset_path = "/home/mindos/data/epiraction_v1.6"
lead_variant_maf = "../../data/lead-maf-vep"


In [3]:
session.spark


In [10]:
epirack = session.spark.read.parquet(dataset_path)
lve = LeadVariantEffect.from_parquet(session=session, path=lead_variant_maf)


In [13]:
# Prefilter lead variants
lve_dataset = lve.maf_filter().effect_size_filter()
# Filter based on score distribution - see 05.0 notebook
epirack_dataset = epirack.filter(f.col("score") > 0.05)


In [ ]:
# Partition both datasets by chromosome and sort by position to ensure rapid range joins
lve_dataset.df.printSchema()


root
 |-- variantId: string (nullable = true)
 |-- studyLocusId: string (nullable = true)
 |-- studyId: string (nullable = true)
 |-- geneId: string (nullable = true)
 |-- originalBeta: double (nullable = true)
 |-- originalStandardError: double (nullable = true)
 |-- pip: double (nullable = true)
 |-- finemappingMethod: string (nullable = true)
 |-- credibleSetSize: integer (nullable = true)
 |-- isTransQtl: boolean (nullable = true)
 |-- variantEffect: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- method: string (nullable = true)
 |    |    |-- assessment: string (nullable = true)
 |    |    |-- score: float (nullable = true)
 |    |    |-- assessmentFlag: string (nullable = true)
 |    |    |-- targetId: string (nullable = true)
 |    |    |-- normalisedScore: double (nullable = true)
 |-- majorLdPopulation: struct (nullable = true)
 |    |-- ldPopulation: string (nullable = true)
 |    |-- relativeSampleSize: double (nullable = true)
 |-- ma